### Importações Necessárias

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
from time import sleep

### Ler a Base de Dados

In [ ]:
df = pd.read_excel('caminho_do_diretorio')
df = pd.read_csv('caminho_do_diretorio')

As próximas linhas de código extrai o ID LATTES a partir da coluna em que estão os links dos currículos dos usuários, adicionando uma nova coluna ao dataframe. O mesmo é feito para o ID INSPER.

<font color="red">IMPORTANTE:</font> Essa parte do código só precisa ser executada caso não tenhamos os IDs LATTES e INSPER na base de dados.

In [7]:
for i in range(0, len(df)):
    if pd.notna(df.loc[i, 'LATTES']):
        id_lattes = df.loc[i, 'LATTES'].split('/')[-1]
        df.loc[i, 'ID LATTES'] = id_lattes
    if pd.notna(df.loc[i, 'INSPER EMAIL']):
        id_insper = df.loc[i, 'INSPER EMAIL'].split('@')[0]
        df.loc[i, 'ID INSPER'] = id_insper

### Criando a função responsável por adicionar os usuários na Plataforma do Insper
```python
adicionar_usuario(link_site, usuario, senha, df, id_coluna_nome, id_coluna_lattes, id_coluna_insper, id_coluna_cpf)
```
- `usuario` (str): O seu usuário Insper.
- `senha` (str): A sua senha de acesso.
- `df` (pandas DataFrame): O DataFrame onde as informações do usuário estão armazenadas.
- `id_coluna_nome` (str): O nome da coluna onde estão os nomes dos usuários a serem cadastrados.
- `id_coluna_lattes` (str): O nome da coluna onde estão os IDs LATTES dos usuários a serem cadastrados.
- `id_coluna_insper` (str): O nome da coluna onde estão os IDs INSPER dos usuários a serem cadastrados.
- `id_coluna_cpf` (str): O nome da coluna onde estão os CPFs dos usuários a serem cadastrados.

In [10]:
# Todos as variáveis dessa função são Strings, menos df, que representa um dataframe
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()

def adicionar_usuario(usuario, senha, df, id_coluna_nome, id_coluna_lattes, id_coluna_insper, id_coluna_cpf):
    # Caso queira que o navegador não abra, basta descomentar a linha abaixo
    # options.add_argument('--headless')
    navegador = webdriver.Chrome(service=service, options=options)

    navegador.get('https://cgi.insper.edu.br/PortfolioDocente/')

    navegador.find_element('xpath', '//*[@id="ContentPlaceHolder1_txtLogin"]').send_keys(usuario)
    navegador.find_element('xpath', '//*[@id="ContentPlaceHolder1_txtSenha"]').send_keys(senha)
    navegador.find_element('xpath', '//*[@id="ContentPlaceHolder1_btnAcessar"]').click()
    sleep(2)
    navegador.find_element('xpath', '//*[@id="ContentPlaceHolder1_btnAdicionar"]').click()
    sleep(2)
    for i in range(0, len(df)):
        nome = df.loc[i, id_coluna_nome]
        id_lattes = df.loc[i, id_coluna_lattes]
        id_insper = df.loc[i, id_coluna_insper]

        # Caso o respectivo usuário não tenha um CPF cadastrado na base de dados, basta adicionar um genérico, já que a plataforma do Insper aceita
        # sem problemas.
        if pd.isna(df.loc[i, id_coluna_cpf]):
            cpf = 83483772049
        else:
            cpf = df.loc[i, id_coluna_cpf]
        navegador.find_element('xpath', '//*[@id="ContentPlaceHolder1_txtNome"]').send_keys(nome) # Nome Completo
        navegador.find_element('xpath', '//*[@id="ContentPlaceHolder1_txtCpf"]').send_keys(cpf) # Cpf
        navegador.find_element('xpath', '//*[@id="ContentPlaceHolder1_txtCodigoLattes"]').send_keys(id_lattes) # Id Lattes
        navegador.find_element('xpath', '//*[@id="ContentPlaceHolder1_txtLogin"]').send_keys(id_insper) # Id Insper
        print(f'Nome: {nome} // CPF: {cpf} // ID Lattes: {id_lattes} // ID Insper: {id_insper} // Linha {i}')
        sleep(1)
        navegador.find_element('xpath', '//*[@id="ContentPlaceHolder1_btnSalvar"]').click()
        sleep(4)

Basta chamar a função na linha de baixo com os parâmetros citados acima

In [ ]:
adicionar_usuario()

### Criando a função responsável por importar os usuários na Plataforma do Insper
```python
importar_usuario(link_site, usuario, senha, df, id_coluna_nome)
```
- `usuario` (str): O seu usuário Insper (Provavelmente você precisará pedir acesso a um funcionário da instituição).
- `senha` (str): A sua senha de acesso.
- `df` (pandas DataFrame): O DataFrame onde as informações do usuário estão armazenadas.
- `id_coluna_nome` (str): O nome da coluna onde estão os nomes dos usuários a serem cadastrados.

In [3]:
# Todos as variáveis dessa função são Strings, menos df, que representa um dataframe
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()

def importar_usuario(usuario, senha, df, id_coluna_nome):
    # Caso queira que o navegador não abra, basta descomentar a linha abaixo
    # options.add_argument('--headless')
    navegador = webdriver.Chrome(service=service, options=options)

    navegador.get('https://cgi.insper.edu.br/PortfolioDocente/')

    navegador.find_element('xpath', '//*[@id="ContentPlaceHolder1_txtLogin"]').send_keys(usuario)
    navegador.find_element('xpath', '//*[@id="ContentPlaceHolder1_txtSenha"]').send_keys(senha)
    navegador.find_element('xpath', '//*[@id="ContentPlaceHolder1_btnAcessar"]').click()
    sleep(2)
    for i in range(0, len(df)):
        nome = df.loc[i, id_coluna_nome]
        print(f'Nome: {nome} // Linha {i}')
        navegador.find_element('xpath', '//*[@id="ContentPlaceHolder1_txtNome"]').send_keys(nome)
        navegador.find_element('xpath', '//*[@id="ContentPlaceHolder1_btnBuscar"]').click()
        sleep(2)
        navegador.find_element('xpath', '//*[@id="ContentPlaceHolder1_gridDocente_chkMarcarTodos"]').click()
        navegador.find_element('xpath', '//*[@id="ContentPlaceHolder1_btnImportar"]').click()
        sleep(5)
        navegador.find_element('xpath', '/html/body/div[1]/div[3]/div/button/span').click()
        navegador.find_element('xpath', '//*[@id="ContentPlaceHolder1_txtNome"]').clear()
        sleep(2)


Basta chamar a função na linha de baixo com os parâmetros citados acima

In [ ]:
importar_usuario()